In [11]:
#required inputs
import pystan
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
import numpy as np

#reading the file from url and droping NaN entries from data
df = pd.read_csv("https://raw.githubusercontent.com/carlson9/KocPython2020/master/homework/trend2.csv").dropna().reset_index(drop=True)


In [12]:
#checking data head
df.head(3)

,country,year,church2,cc,gini_net,rgdpl
0,Australia,1965,26.182790,Australia,29.16514,15.23929
1,Australia,1967,24.744380,Australia,29.50664,16.25018
2,Australia,1979,4.442971,Australia,25.75894,21.58357


In [13]:
#checking data tail
df.tail(3)

,country,year,church2,cc,gini_net,rgdpl
585,United States,2005,31.42520,United States,37.08701,42.53482
586,United States,2006,29.70013,United States,36.87890,43.25840
587,United States,2008,26.67819,United States,36.02488,43.34071


In [15]:
#Getting data into lists
df.columns = df.columns.map(str.strip)
df['country'] = df['country'].str.strip()
noC = df['country'].unique()
cLook = dict(zip(noC, range(len(noC))))
country= df['cc'] = df.c.replace(cLook).values
outcome = df['church2'].values
explanatory = df['gini_net'].values
control = df['rgdpl'].values

In [16]:
my_model = """
data {
    int<lower=0> N; 
    int<lower=0> J; 
    int<lower=1,upper=J> country[N]; 
    vector[N] outc;
    vector[N] ctrl;
    vector[N] expl;
}
parameters {
    vector[J] a;
    real b1;
    real b2;
    real mu_a;
    real<lower=0,upper=100> sigma_a;
    real<lower=0,upper=100> sigma_y;
}
transformed parameters {
    vector[N] predicted;
    vector[N] m;
    for (i in 1:N) {
    predicted[i] = a[country[i]] + expl[i] * b1 + ctrl[i] * b2;
  }
}
model {
    sigma_a ~ uniform(0, 100);
    a ~ normal (mu_a, sigma_a);
    b1 ~ normal (0, 1);
    b2 ~ normal (0, 1);
    sigma_y ~ uniform(0, 100);
    outc ~ normal(predicted, sigma_y);
}
"""

In [18]:
data_my_model ={'N': len(outcome),
                'J': len(df.c),
                'country': country+1,
                'expl': explanatory,
                'ctrl': control,
                'outc': outcome
    
}

In [22]:
my_model_fit = pystan.stan(model_code=my_model, data=data_my_model, iter=1000, chains=2)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_142e540f9be932cc2633ccf5290dd1d8 NOW.


DistutilsPlatformError: Unable to find vcvarsall.bat